# Panel GARCH with Cross Sectional Adjustment

### Az eredeti kódot átdolgoztam, hogy a volatilitás egyenlet könnyebben kezelhető legyen, emellett kijavítottam a beszélt hibákat is, így sokkal gyorsabbá tettem az optimalizálást

In [ ]:
import sys

sys.path.insert(1, 'C:/Users/peter/Desktop/volatility-forecasting/midas')

import numpy as np
import pandas as pd
from base import BaseModel, GarchBase
from stats import loglikelihood_normal, loglikelihood_student_t
from weights import Beta
from helper_functions import create_matrix
from datetime import datetime, timedelta
import time
import scipy.stats as stats

In [15]:
class Panel_GARCH_CSA(BaseModel):
    def __init__(self, plot = True, dist = 'normal', *args):
        self.plot = plot
        self.dist = dist
        self.args = args
    
    def initialize_params(self, X):
        if self.dist == 'normal':
            self.init_params = np.array([0.1, 0.4, 0.4])
        elif self.dist == 'student-t':
            self.init_params = np.array([0.1, 0.4, 0.4, 4.0])
        else:
            raise ValueError("ValueError exception thrown")
        return self.init_params
    
    def model_filter(self, params, X):
        c = np.zeros(X.shape[0])
        sigma2 = np.zeros_like(X)
        
        phi, alpha, beta = params[0], params[1], params[2]
        
        uncond_var = np.nanmean(X ** 2, axis = 0)
        nans = X.isna().sum().values
        X = X.values
        
        for i in range(sigma2.shape[0]):
            if i == 0:
                c[i] = 1
            else:
                c[i] = (1 - phi) + phi * np.nanstd(X[i-1] / (np.sqrt(sigma2[i-1]) * c[i - 1]))
            for j in range(sigma2.shape[1]):
                if nans[j] == i:
                    sigma2[i][j] = uncond_var[j]
                elif nans[j] < i:
                    sigma2[i][j] = uncond_var[j] * (1 - alpha - beta) + alpha * ((X[i - 1][j] / (np.sqrt(sigma2[i-1][j])*c[i-1])) ** 2) + beta * sigma2[i - 1][j]
                else:
                    pass
        return sigma2
    
    def loglikelihood(self, params, X):
        sigma2 = self.model_filter(params, X)
        if self.dist == 'normal':
            lls = loglikelihood_normal(X, sigma2).sum()
        elif self.dist == 'student-t':
            lls = loglikelihood_student_t(X, sigma2, params[3]).sum()
        return lls
    
    def simulate(self, params = [0.1, 0.2, 0.6], num = 100, length = 500):
        c = np.zeros(length)
        sigma2 = np.zeros((length, num))
        ret = np.zeros((length, num))
        
        phi, alpha, beta = params[0], params[1], params[2]
        
        for t in range(length):
            if t == 0:
                c[t] = 1.0
                sigma2[t] = 1.0
            else:
                c[t] = (1 - phi) + phi * np.std(ret[t - 1] / (np.sqrt(sigma2[t - 1]) * c[t - 1]))
                mu = np.mean(ret[ : t] ** 2, axis = 0)
                sigma2[t] = mu * (1 - alpha - beta) + alpha * (ret[t - 1] / (np.sqrt(sigma2[t - 1]) * c[t - 1])) ** 2 + beta * sigma2[t - 1]
            
            ret[t] = stats.norm.rvs(loc = 0.0, scale = np.sqrt(sigma2[t]))
                
        return ret, sigma2, c
    
    def forecast(self, X, H):
        X_new = X
        X_new.loc[X.shape[0]] = 0
        
        sigma2 = self.model_filter(self.optimized_params, X_new)
        sigma2 = sigma2 * np.sqrt(H)
        return sigma2[-1]

In [16]:
model = Panel_GARCH_CSA()

In [17]:
ret, sigma2, c = model.simulate()

In [18]:
model.fit(['01', '01', '01'], pd.DataFrame(ret))

Loglikelihood:  136.44546476393538 

   Parameters  Standard Error  95% CI Lower  95% CI Higher
0    0.000378        2.541696     -4.981255       4.982011
1    0.202491        0.007596      0.187603       0.217378
2    0.588155        0.013644      0.561414       0.614895


# Megvizsgálom a modellt valós adatokkal is, ahol ugyanazokat az időszakokat nézem, mint a Panel_GARCH_Full_vs_Restricted-ben tettem

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from monthdelta import monthdelta

In [23]:
ret_matrix = pd.read_csv('C:/Users/peter/Desktop/volatility-forecasting/results/ret_matrix.csv')
ret_matrix.set_index(pd.to_datetime(ret_matrix.Date), inplace = True)
ret_matrix = ret_matrix.iloc[:, 1:] * 100

ret_mat = ret_matrix.iloc[1:, :]
nan_cols = np.where(ret_mat.isna().sum().values == 1)[0]
nan_index = np.where(ret_mat.iloc[:, nan_cols].isna() == True)[0]
print(set(nan_index))

if len(set(nan_index)) == 1.0:
    ret_mat = ret_mat.drop([ret_mat.index[nan_index[0]]])

{455}


In [24]:
ret_mat_nullnan = ret_mat.iloc[:, np.where(ret_mat.isna().sum().values == 0)[0]]

In [25]:
y = ret_mat[(ret_mat.index >= datetime(1999,12,1) + monthdelta(0)) & (ret_mat.index < datetime(2005,1,1) + monthdelta(0))]
y_nullnan = ret_mat_nullnan[(ret_mat_nullnan.index >= datetime(1999,12,1) + monthdelta(0)) & (ret_mat_nullnan.index < datetime(2005,1,1) + monthdelta(0))]

In [26]:
model = Panel_GARCH_CSA()

# Első lefutás eredményei, mind a szűkített panelra, mind a sztenderdre:

In [27]:
print('---- Teljes Panelra: ----')
model.fit(['01','01', '01'], y)

---- Teljes Panelra: ----


<ipython-input-15-0ecb6a5f99c1>:22: RuntimeWarning: Mean of empty slice
  uncond_var = np.nanmean(X ** 2, axis = 0)
C:/Users/peter/Desktop/volatility-forecasting/midas\stats.py:35: RuntimeWarning: divide by zero encountered in log
  lls = -0.5 * (np.log(2*np.pi) + np.log(sigma2) + resid ** 2 / sigma2)


Loglikelihood:  916.7800055261794 

   Parameters  Standard Error  95% CI Lower  95% CI Higher
0    0.000001        0.402252     -0.788397       0.788400
1    0.122955        0.010429      0.102514       0.143396
2    0.422265        0.023779      0.375659       0.468871


In [28]:
print('---- Szűkebb Panelra: ----')
model.fit(['01', '01', '01'], y_nullnan)

---- Szűkebb Panelra: ----
Loglikelihood:  867.1051005454106 

     Parameters  Standard Error  95% CI Lower  95% CI Higher
0  7.182871e-07        0.424074     -0.831168       0.831170
1  1.240786e-01        0.007533      0.109313       0.138844
2  4.294416e-01        0.015398      0.399262       0.459622


# Második lefutás eredményei, mind a szűkített panelra, mind a sztenderdre a 2008 válság alatt:

In [29]:
y = ret_mat[(ret_mat.index >= datetime(1999,12,1) + monthdelta(40)) & (ret_mat.index < datetime(2005,1,1) + monthdelta(40))]
y_nullnan = ret_mat_nullnan[(ret_mat_nullnan.index >= datetime(1999,12,1) + monthdelta(40)) & (ret_mat_nullnan.index < datetime(2005,1,1) + monthdelta(40))]

In [30]:
print('---- Teljes Panelra: ----')
model.fit(['01','01', '01'], y)

---- Teljes Panelra: ----


<ipython-input-15-0ecb6a5f99c1>:22: RuntimeWarning: Mean of empty slice
  uncond_var = np.nanmean(X ** 2, axis = 0)
C:/Users/peter/Desktop/volatility-forecasting/midas\stats.py:35: RuntimeWarning: divide by zero encountered in log
  lls = -0.5 * (np.log(2*np.pi) + np.log(sigma2) + resid ** 2 / sigma2)


Loglikelihood:  822.0566405111879 

     Parameters  Standard Error  95% CI Lower  95% CI Higher
0  3.617082e-08        0.543502     -1.065244       1.065244
1  6.314596e-02        0.006877      0.049667       0.076625
2  7.172267e-01        0.007630      0.702272       0.732181


In [31]:
print('---- Szűkebb Panelra: ----')
model.fit(['01', '01', '01'], y_nullnan)

---- Szűkebb Panelra: ----
Loglikelihood:  720.1587167981786 

     Parameters  Standard Error  95% CI Lower  95% CI Higher
0  5.263099e-07        0.417410     -0.818109       0.818110
1  6.104480e-02        0.012872      0.035816       0.086273
2  7.459053e-01        0.026033      0.694882       0.796929


# Harmadik lefutás eredményei, mind a szűkített panelra, mind a sztenderdre a 2020 válság alatt:

In [32]:
y = ret_mat[(ret_mat.index >= datetime(1999,12,1) + monthdelta(185)) & (ret_mat.index < datetime(2005,1,1) + monthdelta(185))]
y_nullnan = ret_mat_nullnan[(ret_mat_nullnan.index >= datetime(1999,12,1) + monthdelta(185)) & (ret_mat_nullnan.index < datetime(2005,1,1) + monthdelta(185))]

In [33]:
print('---- Teljes Panelra: ----')
model.fit(['01','01', '01'], y)

---- Teljes Panelra: ----


<ipython-input-15-0ecb6a5f99c1>:30: RuntimeWarning: invalid value encountered in true_divide
  c[i] = (1 - phi) + phi * np.nanstd(X[i-1] / (np.sqrt(sigma2[i-1]) * c[i - 1]))
C:/Users/peter/Desktop/volatility-forecasting/midas\stats.py:35: RuntimeWarning: divide by zero encountered in log
  lls = -0.5 * (np.log(2*np.pi) + np.log(sigma2) + resid ** 2 / sigma2)
<ipython-input-15-0ecb6a5f99c1>:30: RuntimeWarning: invalid value encountered in sqrt
  c[i] = (1 - phi) + phi * np.nanstd(X[i-1] / (np.sqrt(sigma2[i-1]) * c[i - 1]))
<ipython-input-15-0ecb6a5f99c1>:35: RuntimeWarning: invalid value encountered in sqrt
  sigma2[i][j] = uncond_var[j] * (1 - alpha - beta) + alpha * ((X[i - 1][j] / (np.sqrt(sigma2[i-1][j])*c[i-1])) ** 2) + beta * sigma2[i - 1][j]
C:\Users\peter\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:/Users/peter/Desktop/volatility-forecasting/midas\st

Loglikelihood:  6.31533791359477 

   Parameters  Standard Error  95% CI Lower  95% CI Higher
0    0.077387        0.006999      0.063670       0.091105
1    0.535676        0.001996      0.531764       0.539589
2    0.737037        0.007635      0.722073       0.752001


In [34]:
print('---- Szűkebb Panelra: ----')
model.fit(['01', '01', '01'], y_nullnan)

---- Szűkebb Panelra: ----


<ipython-input-15-0ecb6a5f99c1>:30: RuntimeWarning: invalid value encountered in sqrt
  c[i] = (1 - phi) + phi * np.nanstd(X[i-1] / (np.sqrt(sigma2[i-1]) * c[i - 1]))
<ipython-input-15-0ecb6a5f99c1>:35: RuntimeWarning: invalid value encountered in sqrt
  sigma2[i][j] = uncond_var[j] * (1 - alpha - beta) + alpha * ((X[i - 1][j] / (np.sqrt(sigma2[i-1][j])*c[i-1])) ** 2) + beta * sigma2[i - 1][j]
C:\Users\peter\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:/Users/peter/Desktop/volatility-forecasting/midas\stats.py:35: RuntimeWarning: invalid value encountered in log
  lls = -0.5 * (np.log(2*np.pi) + np.log(sigma2) + resid ** 2 / sigma2)
<ipython-input-15-0ecb6a5f99c1>:30: RuntimeWarning: divide by zero encountered in true_divide
  c[i] = (1 - phi) + phi * np.nanstd(X[i-1] / (np.sqrt(sigma2[i-1]) * c[i - 1]))
C:\Users\peter\anaconda3\lib\site-packages\numpy\core\

Loglikelihood:  8.348821348100051 

   Parameters  Standard Error  95% CI Lower  95% CI Higher
0    0.081358        0.006995      0.067647       0.095069
1    0.501210        0.002410      0.496487       0.505933
2    0.691151        0.007804      0.675855       0.706447
